# Lab 5: Mongo and Python with Unbound Data
#### Harrison Listen, Adam Cook, Victoria Feist

In [6]:
from bs4 import BeautifulSoup
import requests
import pgeocode
import pandas as pd
import pymongo

In [7]:
host_name = "localhost"
port = "27017"
conn_str = {f"mongodb://{host_name}:{port}/"}
client = pymongo.MongoClient(conn_str)

### User Instructions
- run the block defining the function
- run the function, then enter a US zip code into the input to receive the current and 7 day weather forecasts, as well as insert these forecasts into a local Mongo database named Weather

In [9]:
def weather():
    user_zip = input("Enter a zip code to retrieve its weather forecast: ")
    us = pgeocode.Nominatim('us')
    lat = str(us.query_postal_code(str(user_zip)).latitude)
    long = str(us.query_postal_code(str(user_zip)).longitude)
    page = requests.get("https://forecast.weather.gov/MapClick.php?lat="+lat+'&lon='+long+"#.Y2wvVOzMI6E")
    soup = BeautifulSoup(page.content, "html.parser")
    seven_day = soup.find(id = "seven-day-forecast")
    period_tags = seven_day.select('.tombstone-container .period-name')
    periods = [pt.get_text() for pt in period_tags]
    short_descs = [sd.get_text() for sd in seven_day.select('.tombstone-container .short-desc')]
    temps = [t.get_text() for t in seven_day.select('.tombstone-container .temp')]
    descs = [d["title"] for d in seven_day.select('.tombstone-container img')]
    weather = pd.DataFrame({
        "Period": periods,
        "Short Description": short_descs,
        "Temp":temps,
        "Description":descs
    })
    current = soup.find('table')
    conditions = [c.get_text() for c in current.find_all('td')]
    current_conditions = pd.DataFrame({
            "Condition": conditions[::2],
            "Current Value": conditions[1::2]
    })
    current_conditions_dict = current_conditions.to_dict(orient = 'list')#turning the dataframes into dicts so they can go into Mongo
    weather_dict = weather.to_dict(orient = 'list')
    db_name = "Weather"
    db = client[db_name]
    sevenday=db.sevenday
    sevenday.insert_one(weather_dict)
    currently = db.currently
    currently.insert_one(current_conditions_dict)
    return weather, current_conditions, "Collections inserted into Mongo database :", db.list_collection_names()
    



In [ ]:
weather()